In [ ]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')

df['is_male'] = df['Gender'].replace({'Female': 0, 'Male' : 1})
df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Gender'], inplace=True)

X = df.drop(columns='Exited')
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion =  df[[ 'HasCrCard', 'IsActiveMember', 'is_male']]
categorical_features = ['Geography']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Obtener los nombres de las columnas después de aplicar OneHotEncoder
encoded_categorical_columns = preprocessor.named_transformers_['cat']\
    .get_feature_names_out(input_features=categorical_features)

# Combinar los nombres de las columnas numéricas y categóricas
processed_columns = numeric_features + list(encoded_categorical_columns)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=processed_columns)
concatenated_series  = pd.concat([processed_df, variables_sin_modificacion.reset_index(drop=True) ], axis=1) # En este código, he utilizado reset_index(drop=True) al obtener variables_sin_modificacion para asegurarme de que los índices coincidan correctamente al realizar la concatenación.

In [ ]:
### MODELO 2 
# Metrica_Modelo	Score
# 38	xgbClassifier_recall_macro	0.868093
# 39	xgbClassifier_precision_macro	0.876113
# 37	xgbClassifier_f1_macro	0.866505
# 36	xgbClassifier_accuracy	0.868107
# 46	SVC_recall_macro	0.792344
# 47	SVC_precision_macro	0.793586
# 45	SVC_f1_macro	0.792119
# 44	SVC_accuracy	0.792346
# 6	Random Forest_recall_macro	0.887075
# 7	Random Forest_precision_macro	0.889157
# 5	Random Forest_f1_macro	0.886920
# 4	Random Forest_accuracy	0.887080
# 2	Logistic Regression_recall_macro	0.709532
# 3	Logistic Regression_precision_macro	0.709742
# 1	Logistic Regression_f1_macro	0.709460
# 0	Logistic Regression_accuracy	0.709535
# 34	LGBclassifier_recall_macro	0.874132
# 35	LGBclassifier_precision_macro	0.886193
# 33	LGBclassifier_f1_macro	0.871222
# 32	LGBclassifier_accuracy	0.874148
# 42	KNN_recall_macro	0.837828
# 43	KNN_precision_macro	0.845589
# 41	KNN_f1_macro	0.836924
# 40	KNN_accuracy	0.837830

# ESTE FUE EL PRIMER RESULTADO DEL CROOS-VALIDATION CON LAS FEATURES NUEVAS LOS DATOS BAJARON BASTANTE Y AHORA ANDAMOS PROBANDO CON TODAS

# Metrica_Modelo	Score
# 38	xgbClassifier_recall_macro	0.851940
# 39	xgbClassifier_precision_macro	0.864105
# 37	xgbClassifier_f1_macro	0.848717
# 36	xgbClassifier_accuracy	0.851956
# 46	SVC_recall_macro	0.750076
# 47	SVC_precision_macro	0.752073
# 45	SVC_f1_macro	0.749600
# 44	SVC_accuracy	0.750079
# 6	Random Forest_recall_macro	0.874214
# 7	Random Forest_precision_macro	0.877441
# 5	Random Forest_f1_macro	0.873911
# 4	Random Forest_accuracy	0.874220
# 2	Logistic Regression_recall_macro	0.666717
# 3	Logistic Regression_precision_macro	0.667271
# 1	Logistic Regression_f1_macro	0.666447
# 0	Logistic Regression_accuracy	0.666721
# 34	LGBclassifier_recall_macro	0.851155
# 35	LGBclassifier_precision_macro	0.865236
# 33	LGBclassifier_f1_macro	0.847113
# 32	LGBclassifier_accuracy	0.851172
# 42	KNN_recall_macro	0.822145
# 43	KNN_precision_macro	0.836495
# 41	KNN_f1_macro	0.820267
# 40	KNN_accuracy	0.822147
# 18	HistGradient_recall_macro	0.845509
# 19	HistGradient_precision_macro	0.856584
# 17	HistGradient_f1_macro	0.842516
# 16	HistGradient_accuracy	0.845524

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pandas as pd

def BaseLine(x_train, y_train, cv: int, metricas_cross_validate: list):
    try:
        modelos = {
            "1": LogisticRegression(),
            "2": RandomForestClassifier(),
            "3": AdaBoostClassifier(),
            "4": GradientBoostingClassifier(),
            "5": ExtraTreesClassifier(),
            "6": DecisionTreeClassifier(),
            "7": CatBoostClassifier(),
            "8": LGBMClassifier(),
            "9": XGBClassifier(),
            "10": KNeighborsClassifier(),
            "11": SVC()
        }

        answer_modelos = input('¿Cuáles son los modelos que desea utilizar? (seleccione números separados por comas o escriba "todos" para seleccionar todos los modelos): 1: Logistic Regression, 2: Random Forest, 3: ADABoosting, 4: GradientBoosting, 5: ExtraTrees, 6: DecisionTree, 7: CatBoost, 8: LGBM, 9: XGBoost, 10: KNN, 11: SVC')

        if answer_modelos.lower() == 'todos':
            modelos_seleccionados = modelos
        else:
            selected_models_indices = [int(x.strip()) for x in answer_modelos.split(',')]
            modelos_seleccionados = {key: modelos[key] for key in map(str, selected_models_indices)}

        metricas = metricas_cross_validate

        resultados_dict = {}

        for nombre_modelo, modelo in modelos_seleccionados.items():
            if cv:
                cv_resultados = cross_validate(modelo, x_train, y_train, cv=cv, scoring=metricas)
            else:
                cv_resultados = cross_validate(modelo, x_train, y_train, cv=5, scoring=metricas)

            for metrica in metricas:
                clave = f"{nombre_modelo}_{metrica}"
                resultados_dict[clave] = cv_resultados[f"test_{metrica}"].mean()

        # Y tienes una lista de nombres que quieres usar para reemplazar los números
        lista_nombres = ['Logistic_Regression', 'Random_Forest', 'ADABoosting', 'Gradient_Boosting', 'Extra_Trees', 'Decision_Tree', 'CatBoost', 'LGBM', 'XGBoost', 'KNN', 'SVC']

        # Utilizamos una comprensión de diccionario para hacer el reemplazo
        diccionario_nombres = {key: lista_nombres[int(key) - 1] for key in modelos_seleccionados}

        resultados_df = pd.DataFrame(resultados_dict.values(), index=diccionario_nombres.values(), columns=["Score"])
        resultados_df.index.name = "Metrica_Modelo"

        return resultados_df.reset_index().sort_values(by="Metrica_Modelo", ascending=False)

    except Exception as e:
        print('Surgió un error:', e)


In [ ]:
# Definir condiciones y valores correspondientes
conditions = [
    (df['Age'] >= 18) & (df['Age'] <= 30),
    (df['Age'] > 30) & (df['Age'] <= 50),
    df['Age'] > 50
]

values = ['Adulto Joven', 'Adulto', 'Adulto Grande']

# Aplicar las condiciones y asignar valores
df['Age_group'] = np.select(conditions, values)


# Gráfico con segmentación por país y género
with plt.style.context('fivethirtyeight'):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 10})
    sns.countplot(data=df, x='Geography', hue='Age_group', palette=['blue', 'pink', 'green'])
    plt.xlabel('Países')
    plt.ylabel('count')
    plt.legend(title='Age_group')
    plt.tight_layout()
    plt.title('Relación entre los países y el grupo de edad')
    plt.show()
    
# Gráfico con segmentación por país y género
with plt.style.context('fivethirtyeight'):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 10})
    sns.countplot(data=df, x='Geography', hue='Gender', palette=['blue', 'pink', 'green'])
    plt.xlabel('Países')
    plt.ylabel('count')
    plt.legend(title='Gender')
    plt.tight_layout()
    plt.title('Relación entre los países y el género')
    plt.show()

# Gráfico con segmentación por país y género
with plt.style.context('fivethirtyeight'):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 10})
    sns.countplot(data=df, x='Geography', hue='Exited', palette=['blue', 'pink', 'green'])
    plt.xlabel('Países')
    plt.ylabel('count')
    plt.legend(title='Exited')
    plt.tight_layout()
    plt.title('Relación entre los países y el abandono hacia el Banco')
    plt.show()
    
# Cálculo del porcentaje de abandono por país y género
result_3 = df.groupby(['Geography', 'Gender', 'Age_group'])['Exited'].value_counts() / len(df) * 100
result_df_3 = result_3.reset_index(name='Porcentaje')
print(result_df_3)

In [ ]:
from scipy.stats import chi2_contingency

# Tabla de contingencia entre 'Geography' y 'Exited'
contingency_table_geo_exited = pd.crosstab(df['Geography'], df['Exited'])

# Prueba de chi-cuadrado entre 'Geography' y 'Exited'
chi2_geo_exited, p_geo_exited, _, _ = chi2_contingency(contingency_table_geo_exited)

# Tabla de contingencia entre 'Gender' y 'Exited'
contingency_table_gender_exited = pd.crosstab(df['Gender'], df['Exited'])

# Prueba de chi-cuadrado entre 'Gender' y 'Exited'
chi2_gender_exited, p_gender_exited, _, _ = chi2_contingency(contingency_table_gender_exited)

# Tabla de contingencia entre 'Age_group' y 'Exited'
contingency_table_age_exited = pd.crosstab(df['Age_group'], df['Exited'])

# Prueba de chi-cuadrado entre 'Age_group' y 'Exited'
chi2_age_exited, p_age_exited, _, _ = chi2_contingency(contingency_table_age_exited)

# Imprimir los resultados
print("Prueba Chi-cuadrado entre Geography y Exited:")
print("Estadístico de Chi-cuadrado:", chi2_geo_exited)
print("Valor p:", p_geo_exited)
print("\nPrueba Chi-cuadrado entre Gender y Exited:")
print("Estadístico de Chi-cuadrado:", chi2_gender_exited)
print("Valor p:", p_gender_exited)
print("\nPrueba Chi-cuadrado entre Age_group y Exited:")
print("Estadístico de Chi-cuadrado:", chi2_age_exited)
print("Valor p:", p_age_exited)
